# 몬테카를로 학습 구현

실제 몬테카를로 prediction을 구현해보자. 목표는 그리드 월드에서 4방향 랜덤 정책의 상태별 가치를 구하는 것이다. 이를 위해 다음 4가지 요소를 구현한다.  
* 환경: 에이전트의 액션을 받아 상태변이를 일으키고, 보상을 줌
* 에이전트: 4방향 랜덤 정책을 이용해 움직임
* 경험 쌓는 부분: 에이전트가 환경과 상호작용하여 데이터를 축적
* 학습하는 부분: 쌓인 경험을 통해 테이블을 업데이트

환경에 특별한 확률적 요소는 없으며, 스텝마다 -1의 보상을 받는다.

## 라이브러리 import

In [1]:
import random

## Grid World 클래스

In [8]:
class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0
    
    def step(self, a):
        if a == 0:
            self.move_right()
        elif a == 1:
            self.move_left()
        elif a == 2:
            self.move_up()
        elif a == 3:
            self.move_down()
        
        reward = -1
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_right(self):
        self.y += 1
        if (self.y > 3):
            self.y = 3
        
    def move_left(self):
        self.y -= 1
        if (self.y < 0):
            self.y = 0
    
    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0
            
    def move_down(self):
        self.x += 1
        if(self.x > 3):
            self.x = 3
    
    def is_done(self):
        if self.x == 3 and self.y == 3:
            return True
        else:
            return False
        
    def get_state(self):
        return (self.x, self.y)
    
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

## Agent 클래스

In [11]:
class Agent():
    def __init__(self):
        pass
    
    def select_action(self):
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action

## Main method

In [16]:
def main():
    env = GridWorld()
    agent = Agent()
    data = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]] # 테이블 초기화
    gamma = 1.0
    alpha = 0.0001
    
    for k in range(50000): # 총 5만번의 에피소드 진행
        done = False
        history = []
        while not done:
            action = agent.select_action()
            (x, y), reward, done = env.step(action)
            history.append((x,y,reward))
        env.reset()
        
        # 에피소드가 끝나면 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0
        for transition in history[:-1]:
            # 방문했던 상태들을 뒤에서부터 보면서 차례대로 리턴을 계산
            x, y, reward = transition
            data[x][y] = data[x][y] + alpha * (cum_reward - data[x][y])
            cum_reward = cum_reward + gamma * reward
            
    # 학습이 끝난 뒤 데이터 출력
    for row in data:
        print(row)

In [17]:
main()

[-37.58329000095512, -42.93231412093926, -50.50630628672253, -54.07970417537915]
[-43.04771291811459, -48.544822610766985, -53.41234727773559, -56.870519850280175]
[-50.44705253526091, -53.28723610920796, -56.049193708760804, -58.233355749196726]
[-54.35467601544412, -55.62130493708832, -57.296167223535285, 0]
